In [10]:
import pandas as pd

# Load data
data = pd.read_csv('output.csv')

# Drop unnecessary columns
data = data.drop(columns=['X', 'Y', 'Z', 'Roll', 'Pitch', 'Yaw'])

# Select features and targets for BaseSpeed and DeltaSpeed models
features_base_speed = data[['Angle']]
target_base_speed = data['BaseSpeed']

features_delta_speed = data[['Error', 'DeltaError']]
target_delta_speed = data['DeltaSpeed']

In [11]:
from sklearn.model_selection import train_test_split

# Split data for BaseSpeed model
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(features_base_speed, target_base_speed, test_size=0.2, random_state=42)

# Split data for DeltaSpeed model
X_train_delta, X_test_delta, y_train_delta, y_test_delta = train_test_split(features_delta_speed, target_delta_speed, test_size=0.2, random_state=42)

In [12]:
from sklearn.linear_model import LinearRegression

# Initialize models
model_base_speed = LinearRegression()
model_delta_speed = LinearRegression()

# Train models
model_base_speed.fit(X_train_base, y_train_base)
model_delta_speed.fit(X_train_delta, y_train_delta)

LinearRegression()

In [13]:
from sklearn.metrics import mean_squared_error, r2_score

# Predictions
y_pred_base = model_base_speed.predict(X_test_base)
y_pred_delta = model_delta_speed.predict(X_test_delta)

# Evaluation for BaseSpeed model
mse_base = mean_squared_error(y_test_base, y_pred_base)
r2_base = r2_score(y_test_base, y_pred_base)
print(f'BaseSpeed - Mean Squared Error: {mse_base}, R-squared: {r2_base}')

# Evaluation for DeltaSpeed model
mse_delta = mean_squared_error(y_test_delta, y_pred_delta)
r2_delta = r2_score(y_test_delta, y_pred_delta)
print(f'DeltaSpeed - Mean Squared Error: {mse_delta}, R-squared: {r2_delta}')

BaseSpeed - Mean Squared Error: 7.020446059922661e-07, R-squared: 0.9999994742874181
DeltaSpeed - Mean Squared Error: 0.2429488949877355, R-squared: 0.06925796527435157


In [14]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {'fit_intercept': [True, False], 'copy_X': [True, False], 'positive': [True, False]}

# Perform Grid Search for BaseSpeed model
grid_search_base = GridSearchCV(model_base_speed, param_grid, cv=5)
grid_search_base.fit(X_train_base, y_train_base)
print("Best parameters for BaseSpeed model:", grid_search_base.best_params_)

# Perform Grid Search for DeltaSpeed model
grid_search_delta = GridSearchCV(model_delta_speed, param_grid, cv=5)
grid_search_delta.fit(X_train_delta, y_train_delta)
print("Best parameters for DeltaSpeed model:", grid_search_delta.best_params_)

Best parameters for BaseSpeed model: {'copy_X': True, 'fit_intercept': True, 'positive': False}
Best parameters for DeltaSpeed model: {'copy_X': True, 'fit_intercept': True, 'positive': True}


In [15]:
# Retrain BaseSpeed model with best parameters
model_base_speed = LinearRegression(**grid_search_base.best_params_)
model_base_speed.fit(X_train_base, y_train_base)

# Retrain DeltaSpeed model with best parameters
model_delta_speed = LinearRegression(**grid_search_delta.best_params_)
model_delta_speed.fit(X_train_delta, y_train_delta)

LinearRegression(positive=True)

In [16]:
import joblib

# Save models
joblib.dump(model_base_speed, 'base_speed_model.pkl')
joblib.dump(model_delta_speed, 'delta_speed_model.pkl')

['delta_speed_model.pkl']

# Training

In [1]:
# Remove duplicate data
import pandas as pd

# Membaca file CSV
file_path = 'output.csv'
df = pd.read_csv(file_path)

# Menampilkan jumlah baris sebelum menghapus duplikat
print(f"Jumlah baris sebelum menghapus duplikat: {df.shape[0]}")

# Menghapus baris yang duplikat
df_cleaned = df.drop_duplicates()

# Menampilkan jumlah baris setelah menghapus duplikat
print(f"Jumlah baris setelah menghapus duplikat: {df_cleaned.shape[0]}")

# Menyimpan kembali file CSV yang sudah bersih dari duplikat
cleaned_file_path = 'cleaned_output.csv'
df_cleaned.to_csv(cleaned_file_path, index=False)


Jumlah baris sebelum menghapus duplikat: 53361
Jumlah baris setelah menghapus duplikat: 53361


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Load data from CSV
data = pd.read_csv('cleaned_output.csv')

# Split data into input and output variables for model 1
X1 = data[['Angle', 'DeltaAngle']]
y1 = data['BaseSpeed']

# Split data into input and output variables for model 2
X2 = data[['Error', 'IntegralError', 'DeltaError']]
y2 = data['DeltaSpeed']

# Split data into training and testing sets
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Support Vector Machine': SVR(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Neural Network': MLPRegressor()
}

# Train and evaluate each model for Task 1
print("Evaluation for Task 1 (BaseSpeed):")
for name, model in models.items():
    model.fit(X1_train, y1_train)
    y1_pred = model.predict(X1_test)
    mse = mean_squared_error(y1_test, y1_pred)
    r2 = r2_score(y1_test, y1_pred)
    print(f"{name}:")
    print(f"  Mean Squared Error: {mse}")
    print(f"  R-squared: {r2}")
    print()
    # Save model
    joblib.dump(model, f'BaseSpeed{name.replace(" ", "")}.joblib')

# Train and evaluate each model for Task 2
print("Evaluation for Task 2 (DeltaSpeed):")
for name, model in models.items():
    model.fit(X2_train, y2_train)
    y2_pred = model.predict(X2_test)
    mse = mean_squared_error(y2_test, y2_pred)
    r2 = r2_score(y2_test, y2_pred)
    print(f"{name}:")
    print(f"  Mean Squared Error: {mse}")
    print(f"  R-squared: {r2}")
    print()
    # Save model
    joblib.dump(model, f'DeltaSpeed{name.replace(" ", "")}.joblib')

Evaluation for Task 1 (BaseSpeed):
Linear Regression:
  Mean Squared Error: 2.2373053908434492e-28
  R-squared: 1.0

Support Vector Machine:
  Mean Squared Error: 0.002416974857501553
  R-squared: 0.9969445212169427

Decision Tree:
  Mean Squared Error: 3.3626254653124726e-08
  R-squared: 0.9999999574905352

Random Forest:
  Mean Squared Error: 2.4939388796432582e-08
  R-squared: 0.9999999684722524

Gradient Boosting:
  Mean Squared Error: 9.430142993922475e-06
  R-squared: 0.9999880786505703

Neural Network:
  Mean Squared Error: 2.451172778488771e-05
  R-squared: 0.9999690128906594

Evaluation for Task 2 (DeltaSpeed):
Linear Regression:
  Mean Squared Error: 5.263111912511625e-30
  R-squared: 1.0

Support Vector Machine:
  Mean Squared Error: 0.0006188787386940545
  R-squared: 0.9982556515769815

Decision Tree:
  Mean Squared Error: 2.750953340204254e-05
  R-squared: 0.9999224626599564

Random Forest:
  Mean Squared Error: 1.755326422280525e-05
  R-squared: 0.999950525027196

Gradien